# SQL Injection URL Detection Model Training

This notebook trains an Isolation Forest model to detect SQL injection attacks in HTTP URLs.

In [ ]:
import sys
from pathlib import Path
import os

# Add project root to sys.path
# This ensures we can import from src/ even if running from training/ subdirectory
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import joblib

from src.features.sql_injection import extract_features

ModuleNotFoundError: No module named 'src'

## Load Training Data

In [ ]:
# Load data
input_csv = '../data/csic_database.csv'
df = pd.read_csv(input_csv, low_memory=False)
print(f"Loaded {len(df):,} rows")

In [ ]:
df.head()

## Load and Extract Features

In [ ]:
# Load data and extract features using shared module
input_csv = '../data/csic_database.csv'
df_raw = pd.read_csv(input_csv, low_memory=False)
print(f"Loaded {len(df_raw):,} rows")

# Extract all SQL injection features
df = extract_features(df_raw)
print(f"Extracted features: {df.columns.tolist()[-7:]}")
print(f"Rule-based detections: {df['is_sqli_flag'].sum():,}")

## Train Isolation Forest Model

In [ ]:
# Prepare features
feature_cols = ['has_sql_keyword', 'sql_keyword_count', 'has_sql_meta',
                'suspicious_param_pattern', 'sql_payload_length', 'sqli_count_60min_user']
feature_cols = [c for c in feature_cols if c in df.columns]

X_raw = df[feature_cols].fillna(0).astype(float).values
print(f"Training with features: {feature_cols}")

# Scale features
scaler = StandardScaler()
X = scaler.fit_transform(X_raw)

# Train model
contamination = 0.02
model = IsolationForest(contamination=contamination, random_state=42)
model.fit(X)
print(f"Model trained with contamination={contamination}")

## Evaluate Model

In [ ]:
# Get predictions
preds = model.predict(X)
df['is_anomaly'] = (preds == -1).astype(int)
df['final_alert'] = ((df['is_sqli_flag'] == 1) | (df['is_anomaly'] == 1)).astype(int)

alerts = df[df['final_alert'] == 1]

print(f"\nResults:")
print(f"Total rows: {len(df):,}")
print(f"Rule-based detections: {df['is_sqli_flag'].sum():,}")
print(f"ML-based detections: {df['is_anomaly'].sum():,}")
print(f"Total alerts: {len(alerts):,}")

In [ ]:
# Save trained model
model_save_path = '../models/sqli_detection_model.joblib'
os.makedirs(os.path.dirname(model_save_path), exist_ok=True)

model_data = {
    'model': model,
    'scaler': scaler,
    'feature_cols': feature_cols,
    'model_type': 'sqli_detection',
    'features_module': 'src.features.sql_injection',  # For dynamic loading in pipeline
    'trained_date': datetime.now().isoformat(),
    'contamination': contamination
}

joblib.dump(model_data, model_save_path)
print(f"Model saved to: {model_save_path}")

## Save Alerts

In [ ]:
# Save alerts
alerts_path = '../output/sqli_alerts.csv'
os.makedirs(os.path.dirname(alerts_path), exist_ok=True)
alerts.to_csv(alerts_path, index=False)
print(f"Alerts saved to: {alerts_path}")